In [16]:
import numpy as np
from math import sqrt, exp

In [17]:
contract="S"
underlying=100.0
strike=100.0
life=365
vol=0.3
rf=0.03
div=0
cp=1
american=True
steps=100

In [18]:
def payoffFunc(und, strk, cp):
    return max((und - strk) * cp, 0)

In [19]:
h = life / 365 / steps
u = exp(vol * sqrt(h))   #upFactor
d = exp(-vol * sqrt(h))  #DownFactor
drift = 1 if (contract == "F") else exp(rf * h)

q = (drift - d) / (u - d)
qx = 1 - q


In [20]:
#build undTree
undTree=[]
for i in range(0,steps+1):
    # Number of prices per period
    period_prices = []
    for j in range(i+1):
        up_factor_repeat = i-j
        down_factor_repeat = j

        values = np.append(np.repeat(u, up_factor_repeat), np.repeat(d, down_factor_repeat))
        assetAtNode = values.prod() * underlying
        period_prices.append(assetAtNode)
    undTree.append(period_prices)

In [21]:
#print (undTree)
#print ("ultimo nivel:",undTree[steps])

In [22]:
opArray=undTree.copy()
#print (opArray)
#print("Ultimo Nivel :",opArray[steps])

In [23]:
#Boundary Condition
for i in range(steps+1):
    opArray[steps][i]=payoffFunc(undTree[steps][i],strike,cp)
    
#print(opArray[steps])


In [24]:
for i in range(steps-1,-1,-1):
    for j in range (len(opArray[i])):
        opAtNode=(opArray[i+1][j+1]*qx+opArray[i+1][j]*q)/drift
        
        if (american == 1):
            opAtNode=max(payoffFunc(undTree[i][j], strike, cp), opAtNode)
        
        opArray[i][j]=opAtNode
    #print(opArray[i]) 
    #print("opArray Final :",opArray)
print("Prima:",opArray[0][0])

Prima: 13.253838123630688


In [25]:
from binomialCRR5 import binomCRR5

In [26]:
contract="S"
underlying=100.0
strike=100.0
life=365
vol=0.3
rf=0.03
div=0
cp=-1
american=True
steps=100


In [27]:
print(binomCRR5(contract,underlying,strike,life,vol,rf,cp,div,american,steps))


10.591711572507526


In [28]:
%timeit binomCRR5(contract,underlying,strike,life,vol,rf,cp,div,american,steps)

1 loop, best of 3: 363 ms per loop
